Nickhil Tekwani CS6220 PROBLEM 4: Knowledge Base Question Answering

In [6]:
## UTILS

import gensim
from nltk import word_tokenize
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict


# FILE GIVEN INCORRECTLY BY ASSIGNMENT, have to use new one
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors

glove_input_file = '/content/glove.6B.50d.txt'
word2vec_output_file = 'glove.6B.50d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

word2vec_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

def get_rel_score_word2vecbase(rel, query):
    if rel not in word2vec_model:
        return 0.0
    words = word_tokenize(query.lower())
    w_embs = []
    for w in words:
        if w in word2vec_model:
            w_embs.append(word2vec_model[w])
    return np.mean(cosine_similarity(w_embs, [word2vec_model[rel]]))


# Function to load the graph from file
def load_graph():
    # Preparing the graph
    graph = defaultdict(list)
    for line in open('/content/graph.txt'):
        line = eval(line[:-1])
        graph[line[0]].append([line[1], line[2]])
    return graph


# Function to load the queries from file
# Preparing the queries
def load_queries():
    queries = []
    for line in open('/content/annotations.txt'):
        line = eval(line[:-1])
        queries.append(line)
    return queries


<ipython-input-6-0a010b009157>:16: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


In [8]:
import gensim
from nltk import word_tokenize
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from sklearn.metrics import f1_score

def find_answers(start_entity, query):
    graph = load_graph()
    visited = set()
    to_visit = [start_entity]
    answer_entities = []

    while to_visit:
        current_entity = to_visit.pop()
        if current_entity in visited:
            continue
        visited.add(current_entity)

        for rel, next_entity in graph.get(current_entity, []):
            # Get score for this relation
            rel_score = get_rel_score_word2vecbase("ns:" + rel, query)
            # A threshold can be introduced here, for simplicity, considering all for now
            to_visit.append(next_entity)

            # As per the problem, the answer is always at the end of the relevant path
            if next_entity not in graph:  # end of the path
                answer_entities.append(next_entity)

    return answer_entities
def set_to_binary_array(s, all_possible_answers):
    return [1 if answer in s else 0 for answer in all_possible_answers]

def evaluate():
    queries = load_queries()

    # Get all possible answers
    all_possible_answers = set()
    for _, _, _, _, _, answers in queries:
        for ans in answers:
            all_possible_answers.add(ans['AnswerArgument'])
    all_possible_answers = list(all_possible_answers)

    y_true = []
    y_pred = []

    for _, query, _, _, _, answers in queries:
        start_entity = query
        true_answer = set([a['AnswerArgument'] for a in answers])
        pred_answer = set(find_answers(start_entity, query))

        y_true.append(set_to_binary_array(true_answer, all_possible_answers))
        y_pred.append(set_to_binary_array(pred_answer, all_possible_answers))

    # Calculating F1 score for each query and averaging
    f1_scores = [f1_score(yt, yp, average='micro') for yt, yp in zip(y_true, y_pred)]

    return np.mean(f1_scores)


print(evaluate())


0.9746835443037973
